In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install flask_restful
!pip install plotly
!pip install pandas

In [4]:
from flask import Flask, render_template, request, send_file
from flask_ngrok import run_with_ngrok
from flask_restful import Api, Resource
import json
import sqlite3
import plotly
import random
import pandas as pd
from io import BytesIO


app = Flask(__name__)
run_with_ngrok(app)
api = Api(app)


class Game(Resource):
    def get(self, game_id):
        q = f'''select * from picks where "index" = ? ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, game_id)
            return json.dumps([dict(zip([key[0] for key in c.description], row)) for row in result])


class GameList(Resource):
    def get(self, start_id, stop_id):
        q = f'''select * from picks where "index" >= ? and "index" <= ? ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, [start_id, stop_id])
            return json.dumps([dict(zip([key[0] for key in c.description], row)) for row in result])


class GamesCount(Resource):
    def get(self):
        q = f'''select count(*) from picks ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q)
            return result.fetchone()[0]


class Heroes(Resource):
    def get(self):
        q = '''select name from heroes ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q)
            return [i[0] for i in result.fetchall()]


class HeroMatches(Resource):
    def get(self, hero):
        q = f'''select * from picks where ? 
IN ("loosing_hero_id_1","loosing_hero_id_2","loosing_hero_id_3", "loosing_hero_id_4", "loosing_hero_id_5", 
"winning_hero_id_1",	"winning_hero_id_2", "winning_hero_id_3",	"winning_hero_id_4", "winning_hero_id_5") ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, [hero])
            return json.dumps([dict(zip([key[0] for key in c.description], row)) for row in result])

class HeroMatchesLoses(Resource):
    def get(self, hero):
        q = f'''select * from picks where ? 
IN ("loosing_hero_id_1","loosing_hero_id_2","loosing_hero_id_3", "loosing_hero_id_4", "loosing_hero_id_5") ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, [hero])
            return json.dumps([dict(zip([key[0] for key in c.description], row)) for row in result])

class HeroMatchesWins(Resource):
    def get(self, hero):
        q = f'''select * from picks where ? 
IN ("winning_hero_id_1","winning_hero_id_2","winning_hero_id_3","winning_hero_id_4","winning_hero_id_5") ;'''
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, [hero])
            return json.dumps([dict(zip([key[0] for key in c.description], row)) for row in result])

class HeroMatchesWinsFriends(Resource):
    def get(self, hero):
        q = 'select "winning_hero_id_1","winning_hero_id_2","winning_hero_id_3","winning_hero_id_4","winning_hero_id_5" from picks where ? IN ("winning_hero_id_1","winning_hero_id_2","winning_hero_id_3","winning_hero_id_4","winning_hero_id_5")'
        with sqlite3.connect("HeroPicks.db") as con:
            c = con.cursor()
            result = c.execute(q, [hero]).fetchall()

            stats = {}

            for i in result:
                for h in i:
                    if h in stats and h != hero:
                        stats[h] += 1
                    else:
                        stats[h] = 1

            return {k: v for k, v in sorted(stats.items(), key=lambda item: item[1], reverse=True)} # отсортировано


api.add_resource(Game, '/api/game/<string:game_id>')
api.add_resource(GameList, '/api/games/<string:start_id>/<string:stop_id>')
api.add_resource(GamesCount, '/api/games/count')
api.add_resource(Heroes, '/api/heroes')
api.add_resource(HeroMatches, '/api/<string:hero>')
api.add_resource(HeroMatchesWins, '/api/<string:hero>/wins')
api.add_resource(HeroMatchesWinsFriends, '/api/<string:hero>/wins/friends')
api.add_resource(HeroMatchesLoses, '/api/<string:hero>/loses')


def paginator_generator(current, maximum, link):
    center = current // 100
    left = center - 5 if center > 5 else 0
    right = center + 5 if center < maximum else maximum

    pages = {}
    for i in range(left, right + 1):
        pages[i] = f"{link}?from={i * 100}&to={i * 100 + 100}"

    return pages


@app.route('/export')
def export():

    data = request.args.get('data', '/')

    if data == '/':
        n = GamesCount().get()
        js = json.loads(GameList().get(0, n))
        df = pd.DataFrame(js, index=[i['index'] for i in js])
        df = df.drop('index', 1)
    else:
        data = data.split('/')[-1]
        js = json.loads(HeroMatches().get(data))
        df = pd.DataFrame(js, index=[i['index'] for i in js])
        df = df.drop('index', 1)

    print('Начинаем генерацию докумета')
    output = BytesIO()
    writer = pd.ExcelWriter(output)

    df.to_excel(writer)
    writer.close()
    output.seek(0)

    print('Документ сгенерирован')

    return send_file(output, attachment_filename="dota_analytics.xlsx", as_attachment=True, cache_timeout=0)


@app.route('/')
def main_page():
    fr = int(request.args.get('from', 0))
    to = int(request.args.get('to', 100))

    n = GamesCount().get()

    graphs = [
        dict(

            data=[
                dict(
                    x=[1, 2, 3],
                    y=[4, 5, 6],
                    type='bar'
                ),
            ],
            layout=dict(
                title='рандомный график'
            )
        ),
    ]

    ids = ['graph-{}'.format(i) for i, _ in enumerate(graphs)]

    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)

    return render_template('template.html', matches=json.loads(GameList().get(fr, to)),
                           pages=paginator_generator(fr, n, '/'), ids=ids,
                           graphJSON=graphJSON,
                           center=fr // 100, heroes=Heroes().get(), title="Общая аналитика")


@app.route('/stats/<string:hero>')
def hero_page(hero):
    fr = int(request.args.get('from', 0))
    to = int(request.args.get('to', 100))

    all = HeroMatches().get(hero)
    wins = HeroMatchesWins().get(hero)
    loses = HeroMatchesLoses().get(hero)

    win_friends = HeroMatchesWinsFriends().get(hero)

    graphs = [
        dict(

            data=[
                dict(
                    x=['Wins', 'Loses'],
                    y=[len(wins), len(loses)],
                    type='bar'
                ),
            ],
            layout=dict(
                title='Победы / поражения'
            )
        ),

        dict(
            data=[
                dict(
                    x=list(win_friends.keys()),
                    y=list(win_friends.values()),
                    type='bar'
                ),
            ],
            layout=dict(
                title='Лучшие друзья'
            )
        ),

    ]

    ids = ['graph-{}'.format(i) for i, _ in enumerate(graphs)]

    graphJSON = json.dumps(graphs, cls=plotly.utils.PlotlyJSONEncoder)

    return render_template('template.html', matches=json.loads(all)[fr:to],
                           heroes=Heroes().get(), ids=ids,
                           graphJSON=graphJSON, title="Аналитика героя " + hero.title(),
                           pages=paginator_generator(fr, len(all), f'/stats/{hero}'),
                           center=fr // 100
                           )


app.run()
